In [2]:
import pandas as pd
from astropy.io import fits
import numpy as np
from pathlib import Path
import glob




In [ ]:
# MUST BE RUN IN ASTROCONDA38

from fast_combine import fast_process, marckie_slice_combine_reduce

In [4]:
%mkdir processed
%mkdir processed_uncorrected

dark_hdu = fits.open(Path('./calibs/Master_DarkBias.fits'))
#SuperDarkBias = dark_hdu[0].data
SuperDarkBias = dark_hdu[0].data.astype(np.float32)


flatC_norm_hdu = fits.open(Path('./calibs/Master_Flat_C_norm.fits'))
SuperNormFlat_C = flatC_norm_hdu[0].data.astype(np.float32)


mkdir: processed: File exists
mkdir: processed_uncorrected: File exists


In [4]:
fast_process(glob.glob('MBAFiles/*/*'), './processed/', exp_time=10, chunk_size=6,
    darkbias=SuperDarkBias, flat=SuperNormFlat_C, do_offset=True
    )

starting process
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_1.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_0.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_1.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_0.fits
combined_2022-11-12T07:22:56.0000__Brorfelde_4_20221111_224933_1.fits
reducing image 0 from file Brorfelde_4_20221111_224933_1.fits with timestamp offset 2000
Successfully sliced and combined combined_2022-11-12T07:22:56.0000__Brorfelde_4_20221111_224933_1.fits
combined_2022-11-12T03:11:08.0000__Brorfelde_1_20221111_183745_1.fits
reducing image 0 from file Brorfelde_1_20221111_183745_1.fits with timestamp offset 2000
combined_2022-11-1

In [3]:
fast_process(glob.glob('MBAFiles/*/*'), './processed_uncorrected/', exp_time=10, chunk_size=6,
    darkbias=0, flat=1, do_offset=True
    )

starting process
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_0.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_1.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_1.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_0.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_1.fits
combined_2022-11-12T04:27:31.0000__Brorfelde_2_20221111_195408_1.fits
reducing image 0 from file Brorfelde_2_20221111_195408_1.fits with timestamp offset 2000
combined_2022-11-12T03:11:08.0000__Brorfelde_1_20221111_183745_1.fits
reducing image 0 from file Brorfelde_1_20221111_183745_1.fits with timestamp offset 2000
combined_2022-11-12T05:46:20.0000__Brorfelde_3_20221111_211257_1.fits
reducing image 0 from file Brorfelde_3_20221111_211

In [ ]:
%mkdir combined_all

fast_process(glob.glob('MBAFiles/*/*'), './combined_all/', exp_time=10, chunk_size=6,
    darkbias=SuperDarkBias, flat=SuperNormFlat_C, do_offset=True
    )

In [3]:
# MUST BE RUN IN (astroconda3)

import alipy
from datetime import datetime

from dateutil.parser import parse


In [4]:
%mkdir aligned

def align_imgs(images_to_align, ref_image):

    identifications = alipy.ident.run(ref_image, list(images_to_align), visu=False)

    outputshape = alipy.align.shape(ref_image)

    for id in identifications: # list of the same length as images_to_align.
        if id.ok == True:
            alipy.align.affineremap(id.ukn.filepath, id.trans,
                shape=outputshape,
                makepng=True,
                outdir=f'aligned/')

            print("SUCCESS: %20s : %20s, flux ratio %.2f" % (id.ukn.name,
                                                        id.trans,
                                                        id.medfluxratio))

        else:
            print("FAILED: %20s" % (id.ukn.name))
    return identifications

mkdir: aligned: File exists


In [9]:
def _get_time(fp:str):
    name = Path(fp).name.split('__')[0].lstrip('combined_')
    return parse(name)
    #return datetime.datetime.strptime(name, 'combined_%Y-%m-%dT%X.0000')


imgs = pd.DataFrame(
    columns=('filepath', 'timestamp', 'ref_img'),
    #dtype=(str, datetime, str)
)

imgs['filepath'] = glob.glob(f"processed/*.fits")

imgs['timestamp'] = imgs['filepath'].apply(_get_time)

night_1 = (imgs['timestamp'] >= parse('2022-11-11T05:51:00')) & (imgs['timestamp'] < parse('2022-11-12T00:00:00'))

imgs.loc[night_1, 'ref_img'] = './processed/combined_2022-11-11T05:51:00.0000__Brorfelde_4_20221110_215057_0.fits'


night_2_group_1 = (
    (parse('2022-11-12T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-12T05:59:20'))
    )

night_2_group_2 = (
    (parse('2022-11-12T05:59:20') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-13T00:00:00'))
    )

imgs.loc[night_2_group_1, 'ref_img'] = './processed/combined_2022-11-12T04:36:31.0000__Brorfelde_2_20221111_195408_1.fits'
imgs.loc[night_2_group_2, 'ref_img'] = './processed/combined_2022-11-12T06:57:36.0000__Brorfelde_4_20221111_224933_0.fits'

night_3 = (
    (parse('2022-11-13T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-14T00:00:00'))
    )

imgs.loc[night_3, 'ref_img'] = ('./processed/combined_2022-11-13T02:40:26.0000__Brorfelde_1_20221112_180823_0.fits')

imgs.dropna(inplace=True)

imgs[night_3]



# times = {datetime.datetime.strptime()}
# combine_imgs()

/Users/student/anaconda3/envs/astroconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,filepath,timestamp,ref_img
0,processed/combined_2022-11-13T03:49:33.0000__B...,2022-11-13 03:49:33,./processed/combined_2022-11-13T02:40:26.0000_...
7,processed/combined_2022-11-13T03:51:33.0000__B...,2022-11-13 03:51:33,./processed/combined_2022-11-13T02:40:26.0000_...
11,processed/combined_2022-11-13T02:31:26.0000__B...,2022-11-13 02:31:26,./processed/combined_2022-11-13T02:40:26.0000_...
12,processed/combined_2022-11-13T03:35:33.0000__B...,2022-11-13 03:35:33,./processed/combined_2022-11-13T02:40:26.0000_...
15,processed/combined_2022-11-13T02:12:26.0000__B...,2022-11-13 02:12:26,./processed/combined_2022-11-13T02:40:26.0000_...
...,...,...,...
651,processed/combined_2022-11-13T05:08:40.0000__B...,2022-11-13 05:08:40,./processed/combined_2022-11-13T02:40:26.0000_...
652,processed/combined_2022-11-13T05:38:00.0000__B...,2022-11-13 05:38:00,./processed/combined_2022-11-13T02:40:26.0000_...
655,processed/combined_2022-11-13T04:17:53.0000__B...,2022-11-13 04:17:53,./processed/combined_2022-11-13T02:40:26.0000_...
657,processed/combined_2022-11-13T06:35:57.0000__B...,2022-11-13 06:35:57,./processed/combined_2022-11-13T02:40:26.0000_...


In [10]:
from os.path import exists
# IF FILE EXISTS
imgs['ref_img'].apply(exists).all()

True

In [12]:
for group in (night_1,):
    identifications = align_imgs(imgs[group]['filepath'], imgs[group]['ref_img'].iloc[0])
    imgs.loc[group, 'ok'] = [id.ok for id in identifications]

/Users/student/anaconda3/envs/astroconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


##########  Preparing reference ...
No filter file found, using default filter
No NNW file found, using default NNW config
Number of sources in catalog : 6
EXT_NUMBER values found in catalog : 1
I've selected 6 sources
Making more quads, from quadlevel 0 ...
Made   15 quads from    6 stars (combi n=7 s=0 d=30.0)
Removing 0/15 duplicates
########## Processing processed/combined_2022-11-11T07:16:31.0000__Brorfelde_5_20221110_225228_0.fits
No filter file found, using default filter
No NNW file found, using default NNW config
Number of sources in catalog : 1
EXT_NUMBER values found in catalog : 1
I've selected 1 sources
Not enough stars in the unknown catalog.
No matching stars to compute flux ratio !
########## Processing processed/combined_2022-11-11T06:14:00.0000__Brorfelde_4_20221110_215057_0.fits
No filter file found, using default filter
No NNW file found, using default NNW config
Number of sources in catalog : 6
EXT_NUMBER values found in catalog : 1
I've selected 6 sources
Making m

In [35]:
#success rate night 3
sum(imgs[night_3]['ok']) / imgs[night_3].shape[0]

0.6229508196721312